# LSTM

- ## Preliminaries

- ### Imports

In [1]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [2]:
get_available_gpus()

[u'/gpu:0']

- ### Test

In [4]:
%%time
'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

corpus length: 600901
total chars: 59
nb sequences: 200287
Vectorization...
Build model...

--------------------------------------------------
Iteration 1
Epoch 1/1
200287/200287 [==============================] - 244s - loss: 1.9739   

----- diversity: 0.2
----- Generating with seed: "utwitting: in fact, he perceived
the irr"
utwitting: in fact, he perceived
the irressing and the sack of the precess of the special the spect of the condence of the sack of the and the sach and and a the are and the have the are as the sack of the sastions and the are as the precess of the sack of the are and the spirit of the man the precess and all the sack of the some the existed to the sack of the man the and and a the precess of the sack of the are the precess of the such 

----- diversity: 0.5
----- Generating with seed: "utwitting: in fact, he perceived
the irr"
utwitting: in fact, he perceived
the irresurit of the mas to the man as it is call the precess to the that the symmonions, as which the 

the meid if agree, upon nature-daiging
of religious who
receatue fer, fors-lipucy--and all
this relatedy) cause comalitudeots the pislect, and pact re aque, the moke

--------------------------------------------------
Iteration 8
Epoch 1/1
200287/200287 [==============================] - 250s - loss: 1.4158   

----- diversity: 0.2
----- Generating with seed: ", we should do honour to
our name--we ar"
, we should do honour to
our name--we are not the probable to the possibility, and the probable to the present the consciences of the existence of the strongly the probleme of the present the present in the probable the same power of the probably the probable the present of the probable the problace that it is the present of the senses and consciences in the possibility, and with the probable that it is the philosophy of the present of 

----- diversity: 0.5
----- Generating with seed: ", we should do honour to
our name--we ar"
, we should do honour to
our name--we are and most profound t

"history endly alsoutt ispon thing dockreglud, from "things of them as the baring of all and fand his, . "something of
his good and of rulefunt as
men; one mediting; the illagnous, he
is they have been
beyond oicfuld drifw; ther; is 

--------------------------------------------------
Iteration 15
Epoch 1/1
200287/200287 [==============================] - 243s - loss: 1.3615   

----- diversity: 0.2
----- Generating with seed: "the
strict usages of your ears? certainl"
the
strict usages of your ears? certainly the conscience of the such a standing the such a such philosopher to be such a principal and desire and consequently and process and belief in the sense of the souls at last and the properted and conscience of the more process of the subsect and the stated the conscience of the subtlety and the stated the sense of the soul and for the sense, the power and the power of the power of the probably a

----- diversity: 0.5
----- Generating with seed: "the
strict usages of your ears? ce

200287/200287 [==============================] - 242s - loss: 1.3334   

----- diversity: 0.2
----- Generating with seed: "nd scorn, which can no longer endure the"
nd scorn, which can no longer endure the power to the such an acts and considerations of the property, as a concerned in the consequently all the senses of the delight of the an amance of the sense of the states. the present in the property, and the seriousness of the propes the present and soul of the same to the property, as the such an acts of the restlated to the possible, and in the seriousness of the such an its instinctive and th

----- diversity: 0.5
----- Generating with seed: "nd scorn, which can no longer endure the"
nd scorn, which can no longer endure the religion that it was delight undiscomptive men and
stoing of the european the weard hard with all the struction. in the sense" as in a proper in the presented to the other things in the consequently all the such period of all and all as all the consideration i

/usr/local/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in log


...

not havideance to themselves are as pity here are stupeful thance hitherto peaciable
very
indevire to a strengthong sided to songe-thus and kism. but not always of the frout: " from all narrou

----- diversity: 1.2
----- Generating with seed: "ss, and cupidity. it
might even be possi"
ss, and cupidity. it
might even be possible simpter their scienief.t sappss this ba freedom from the wroture, fraisity, that word when observations of whom life
furthy, betrozaris, have not here a ortaniaty,
libebares wouh
still who alone thus have more furths all did of read, is most
astenas would question of value of europe not be be nookhing calls irreapon whatever--it is questions, etrecences for even and saperes! what reation. a co

--------------------------------------------------
Iteration 32
Epoch 1/1
200287/200287 [==============================] - 242s - loss: 1.3444   

----- diversity: 0.2
----- Generating with seed: "ophy become thus a tragedy?
will not tru"
ophy become thus a tragedy?
will 

have empain: there obedient moftachershady. noce and making the        eg "knowpen. thus here it is but will as screct who have of wome trmoness is the to armohe it be
thour antimage to entirelt all the insifition. would not imeapers of the eefiri whewer
co

----- diversity: 1.2
----- Generating with seed: "destiny of the world and, notwithstandin"
destiny of the world and, notwithstanding
kind of man spoing riguti the enemity gratifuy are one labour, more instance, un        

       s hothheque: that in rejomable and weok: yould hone--follow, out of advontant yel) ourselves.

2nee my lack sedectms of the busition to ?mlend of mugityed thing along out of a takeon which, more
natured.ly ivinity; pul if up feries from it.les
is to a dungant out of
pettern encuten
that abohe order n

--------------------------------------------------
Iteration 39
Epoch 1/1
200287/200287 [==============================] - 242s - loss: 1.4323   

----- diversity: 0.2
----- Generating with seed: "lar philos

is for td emandualitaer ownisuicaninn gn
therrow. his without is so an are enduce must mistid in the begnished fromge, ourtaryd
ufil are gict
ermor,. "t

----- diversity: 1.2
----- Generating with seed: "s in the power of science to release us "
s in the power of science to release us react itdbly
uptlar apprievorilye boinviel indendobrtious
i taep or
in theat iunly out
mentruyly clifaal
ehojp effree suf kalmyis, and caseerde, to plasmest of is if mory, inndiectiveryehond., to onces hely, hithiuse,s of cunsiskeing that ever that curat-"unof that of pasate sany ald to
is demod fasm." ycond motaeniustidn hadani fratiorued 
of bet
therow
is andly itdvonac
nuhlulningliriy fooly fro

--------------------------------------------------
Iteration 46
Epoch 1/1
200287/200287 [==============================] - 243s - loss: 2.0069   

----- diversity: 0.2
----- Generating with seed: "r the mask of philanthropy--and precisel"
r the mask of philanthropy--and precisely of the sodiment the protuthene 

tht is onde neeiciloinoes sals the. thenlersrigootsg, in urt anosutiseesoth uonse anean optthhht opleudo thecisdinthel acng is, ese    3entes sok secdsatioulinsait oultto 
allionc. ufes b ch ituanteiaseye ia voil   ooe irdee ljdat.l moeggintate rou lith shi for tyiheeng eavyt hethantct assie, ofeee thy s

----- diversity: 1.2
----- Generating with seed: "-this word is not to be taken too strict"
-this word is not to be taken too strict foewoi anwocm ryn dosigoeedy hiyn s vhf inleiruantiie.satencuthices thepe and hssopfunryl ome mabe
on i diia    aamyt virt
tat athne, th
hhlubhine susuil
theenisdw ge everhir sh ingge haasdislusuwthutflosm uuipdlpovhftn rvieat be madhtutthot ho mh m esd ther c dlesee:sy th elt  toabfithsednaslilfosofaogs   a eleand " than,--espot s m thr-,
ce rat krids,anli gllo
is cussiaiodhhp.niwordyng.pieicanl

--------------------------------------------------
Iteration 53
Epoch 1/1
200287/200287 [==============================] - 243s - loss: 3.3467   

----- divers

 oed te,id p haxaf iad"t s dadorsin ths th
o tha"st "i  fjeeans rxesuoit and.athea, in bll aespd setasrh" enno"t.ino nmsusid she eens- vortecnggofo thi se sqdhofbero hi 

----- diversity: 1.2
----- Generating with seed: "nesty, is probably the
most dangerous an"
nesty, is probably the
most dangerous an ekdeytpiemoyass ailingutishen e
r
ectha f"i thyps brabaigooogplatc rooiy  imr ser ilatant" fcrhar tofbool wfeltt lr iaoit heplorendjt en t bns mohndinlinbtantlelein th mi poc"ftringtrif ypg
  timiounrefhaieyth m co tilleynl se, thomotso sholltguinwirn  
y a wqimithetawhodakeyignueg ci frdifincrmlisncat"
hutt s pxiffxgs id"teeenlaon anhyioep it i   hylw 
=scrosr s ehac cs ax.sdrltm thiffdwey
nisgi
CPU times: user 17h 39min 51s, sys: 6h 57min 3s, total: 1d 36min 55s
Wall time: 4h 25min 59s
